In [1]:
!nvidia-smi

Tue Jul  8 16:56:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   54C    P8             N/A /  115W |     851MiB /   8188MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# 📌 1. 기본 라이브러리
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, f1_score
from xgboost import XGBClassifier
from collections import Counter

ModuleNotFoundError: No module named 'xgboost'

In [5]:
# 📌 2. 데이터 불러오기
train = pd.read_csv('피처_40/train_40.csv')
train

,ID,Segment,입회일자_신용,탈회횟수_발급1년이내,_1순위카드이용건수,_2순위카드이용금액,최종카드발급경과월,CA한도금액,월상환론한도금액,카드이용한도금액_B2M,...,연체입금원금_B5M,연속유실적개월수_기본_24M_카드,이용금액대,최종이용일자_CA_년,청구금액_B0,청구금액_R6M,할인금액_청구서_B0M,잔액_일시불_B0M,평잔_CA_해외_3M,평잔_일시불_6M
0,TRAIN_000000,D,20130101,0,26,0,22,7270,0,19723,...,5752,13,120,2018,12226,88693,0,998,0,2440
1,TRAIN_000001,E,20170801,0,46,0,18,5718,90611,9998,...,821,12,40,2017,5834,16861,163,2565,0,2677
2,TRAIN_000002,C,20080401,0,28,0,20,35207,0,77975,...,7014,8,120,2018,21866,165221,0,5312,0,9118
3,TRAIN_000003,D,20160501,0,1,0,17,6531,0,19226,...,11196,5,120,2018,16356,127371,0,730,0,884
4,TRAIN_000004,E,20180601,0,-2,0,15,47149,155020,168681,...,0,0,0,0,0,155,0,0,0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,TRAIN_399995,E,20010701,0,3,0,39,10167,54445,21152,...,0,0,0,0,0,0,0,0,0,0
2399996,TRAIN_399996,D,20170701,0,38,0,24,31159,156800,78140,...,1910,17,120,0,14402,99849,0,3351,0,12524
2399997,TRAIN_399997,C,20090501,0,33,0,18,19429,0,63374,...,856,24,75,0,5731,41073,0,2524,0,3241
2399998,TRAIN_399998,E,20130101,0,-2,0,27,4228,180906,10001,...,507,0,0,0,0,0,0,0,0,0


In [6]:
test = pd.read_csv('피처_40/test_40.csv')
test

,ID,입회일자_신용,탈회횟수_발급1년이내,_1순위카드이용건수,_2순위카드이용금액,최종카드발급경과월,CA한도금액,월상환론한도금액,카드이용한도금액_B2M,이용금액_체크_R12M,...,연체입금원금_B5M,연속유실적개월수_기본_24M_카드,이용금액대,최종이용일자_CA_년,청구금액_B0,청구금액_R6M,할인금액_청구서_B0M,잔액_일시불_B0M,평잔_CA_해외_3M,평잔_일시불_6M
0,TEST_00000,20140501,0,51,6021,45,18131,0,50006,1752,...,2414,5,75,0,4931,22151,0,4802,0,2933
1,TEST_00001,20160201,0,40,0,19,16819,0,50003,46255,...,2959,8,75,2017,10152,32878,14,1201,0,657
2,TEST_00002,20180301,0,154,10704,5,30505,19499,100056,1004,...,0,24,120,0,13223,71867,50,3046,0,6253
3,TEST_00003,20120701,0,105,0,7,6402,156102,19693,-414,...,0,5,20,2015,2112,4986,0,768,0,819
4,TEST_00004,20031201,0,52,6387,1,0,52784,3924,6412,...,0,20,40,2013,4406,10758,0,637,0,979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,TEST_99995,20130301,0,-2,0,0,0,0,0,1168,...,0,0,0,0,0,0,0,0,0,0
599996,TEST_99996,20171001,0,4,0,12,17876,152797,49990,-414,...,689,8,5,0,359,2237,50,191,0,137
599997,TEST_99997,20180601,0,6,0,11,13332,156313,30004,1394,...,0,0,0,0,0,0,0,0,0,0
599998,TEST_99998,20120201,0,185,31916,1,17362,204480,37139,81487,...,12448,24,120,0,21273,108420,242,17916,0,18108


In [7]:
# ▶ 1. 피처(X), 정답(y) 분리
X_vif = train.drop(columns=['Segment'])
y_vif = train['Segment']

# ▶ 2. 필요 시 레이블 인코딩 (CatBoost는 문자형도 처리 가능하지만 교차검증 편의 위해 사용)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_vif_encoded = le.fit_transform(y_vif)

In [8]:
# 'ID' 컬럼 제거
X_vif = X_vif.drop(columns=['ID'], errors='ignore')

In [9]:
X_vif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 40 columns):
 #   Column              Dtype
---  ------              -----
 0   입회일자_신용             int64
 1   탈회횟수_발급1년이내         int64
 2   _1순위카드이용건수          int64
 3   _2순위카드이용금액          int64
 4   최종카드발급경과월           int64
 5   CA한도금액              int64
 6   월상환론한도금액            int64
 7   카드이용한도금액_B2M        int64
 8   이용금액_체크_R12M        int64
 9   최대이용금액_일시불_R12M     int64
 10  최대이용금액_체크_R12M      int64
 11  이용개월수_할부_무이자_R3M    int64
 12  쇼핑_도소매_이용금액         int64
 13  _1순위업종_이용금액         int64
 14  _2순위업종_이용금액         int64
 15  _3순위업종_이용금액         int64
 16  _2순위쇼핑업종_이용금액       int64
 17  _3순위쇼핑업종_이용금액       int64
 18  _1순위교통업종_이용금액       int64
 19  할부금액_무이자_3M_R12M    int64
 20  이용금액_오프라인_R6M       int64
 21  이용건수_오프라인_R6M       int64
 22  이용금액_오프라인_R3M       int64
 23  이용금액_오프라인_B0M       int64
 24  이용금액_A페이_R3M        int64
 25  이용금액_선결제_B0M        int64
 26  연체입금원금_B0M    

In [ ]:
# ▶ CatBoost 설치
!pip install -q catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 24.6 MB/s eta 0:00:00


In [10]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# ▶ 교차검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = []

# ▶ CatBoost 파라미터
model_params = {
    "loss_function": "MultiClass",      # 다중 클래스용
    "eval_metric": "MultiClass",
    "task_type": "GPU",
    "learning_rate": 0.01,
    "iterations": 70000,
    "l2_leaf_reg": 50,
    "random_seed": 42,
    "od_type": "Iter",
    "depth": 5,
    "early_stopping_rounds": 15000,
    "border_count": 64,
    "verbose": 0
}

# ▶ 교차검증 반복
for fold, (train_idx, val_idx) in enumerate(cv.split(X_vif, y_vif_encoded)):
    print(f"▶ Fold {fold + 1}")

    X_train, X_val = X_vif.iloc[train_idx], X_vif.iloc[val_idx]
    y_train, y_val = y_vif_encoded[train_idx], y_vif_encoded[val_idx]

    train_pool = Pool(data=X_train, label=y_train)
    val_pool = Pool(data=X_val, label=y_val)

    model = CatBoostClassifier(**model_params)
    model.fit(train_pool, eval_set=val_pool, use_best_model=True)

    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred, average='micro')
    f1_scores.append(f1)

    print(f"✅ F1_micro: {f1:.5f}")

# ▶ 평균 F1 출력
print(f"\n🎯 평균 F1_micro (5 folds): {np.mean(f1_scores):.5f}")

▶ Fold 1
✅ F1_micro: 0.91040
▶ Fold 2
✅ F1_micro: 0.90989
▶ Fold 3
✅ F1_micro: 0.91035
▶ Fold 4
✅ F1_micro: 0.91093
▶ Fold 5
✅ F1_micro: 0.91038

🎯 평균 F1_micro (5 folds): 0.91039


In [14]:
# ▶ 전체 데이터로 최종 모델 학습
train_pool_all = Pool(data=X_vif, label=y_vif_encoded)
final_model = CatBoostClassifier(**model_params)
final_model.fit(train_pool_all, use_best_model=False)  # eval_set이 없으므로 early_stopping은 생략

In [ ]:
# ▶ cat_features 정의 (범주형...없긴 함)
cat_features = X_vif.select_dtypes(include='object').columns.tolist()

In [15]:
import pickle
import pandas as pd
import numpy as np

# ▶ 저장 경로 지정
base_path = '피처_40'

# ▶ 1. Fold별 f1_micro 점수 저장
with open(f'{base_path}/f1_scores.pkl', 'wb') as f:
    pickle.dump(f1_scores, f)

# ▶ 2. 평균 f1_micro 점수 저장
with open(f'{base_path}/f1_mean.txt', 'w') as f:
    f.write(str(np.mean(f1_scores)))

# ▶ 3. LabelEncoder 저장 (Segment 인코딩 복원용)
with open(f'{base_path}/label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

# ▶ 4. 학습용 피처(X_vif)와 타깃(y_vif_encoded) 저장
X_vif.to_csv(f'{base_path}/X_vif.csv', index=False)
pd.Series(y_vif_encoded, name='Segment_encoded').to_csv(f'{base_path}/y_vif_encoded.csv', index=False)

# ▶ 5. 범주형 컬럼 리스트 저장
# with open(f'{base_path}/cat_features.pkl', 'wb') as f:
#     pickle.dump(cat_features, f)

# ▶ 6. 최종 모델 저장 (선택)
final_model.save_model(f'{base_path}/final_catboost_model.cbm')


In [ ]:
# from xgboost import XGBClassifier
# from sklearn.model_selection import StratifiedKFold, cross_val_score
# from sklearn.metrics import make_scorer, f1_score

# # 클래스 비율 기반 sample_weight 계산
# # → 이미 y_encoded 가 숫자형이면 그대로 사용
# from collections import Counter

# class_counts = Counter(y_encoded)
# total_count = sum(class_counts.values())
# class_weights = {cls: total_count / count for cls, count in class_counts.items()}

# # 각 샘플에 대한 가중치 부여
# sample_weights = np.array([class_weights[cls] for cls in y_encoded])

# # XGBoost 모델 정의 (GPU 사용)
# model = XGBClassifier(
#     objective='multi:softprob',
#     num_class=5,  # 클래스 수 (A~E)
#     tree_method='gpu_hist',  # ✅ GPU 사용
#     eval_metric='mlogloss',
#     use_label_encoder=False,
#     n_jobs=-1,
#     random_state=42
# )

# # Stratified K-Fold 설정
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # f1_micro scorer 정의
# f1_micro = make_scorer(f1_score, average='micro')

# # 교차검증 수행 (sample_weight 사용)
# scores = cross_val_score(
#     model,
#     X, y_encoded,
#     cv=cv,
#     scoring=f1_micro,
#     fit_params={'sample_weight': sample_weights}
# )

# print(f"XGBoost f1_micro 평균 점수: {scores.mean():.4f}")